# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [33]:
from random import random, seed
import math
from itertools import product
import numpy as np

from icecream import ic

## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

In [34]:
UNIVERSE_SIZE = 1000
NUM_SETS = 100
DENSITY = 0.2 #how dense are the sets, how many elements are covered by each set


rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))

In [35]:
# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.pow(SETS.sum(axis=1), 1.1)

## Helper Functions

In [36]:
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution]))


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()


In [37]:
#The fitness function evaluate the goodness of a solution
def fitness(solution: np.ndarray):
    return (valid(solution), -cost(solution))

## Optimized (somehow clever) starting point computation 

In [38]:
#is it good to start from a smarter solution?
#It could be, but then you could find yourself stuck close to a local optima
#Let's try with a smarter starting point
#Let's first select critical sets, those that contains an element that is not covered by any other set

is_critical = np.zeros(NUM_SETS, dtype=bool) #It could be also used later to avoid removing critical sets
for element in range(UNIVERSE_SIZE):
    sets_covering_element = np.where(SETS[:, element])[0]
    if len(sets_covering_element) == 1:  # If the element is covered by only one set
        is_critical[sets_covering_element[0]] = True

starting_optimized = np.full(NUM_SETS, False)

starting_optimized[is_critical] = True #We have first added critical sets to the solution

#COSTS are sorted in increasing order, so we can take the sets with the lowest cost until the solution is not valid

sorted_indices = np.argsort(COSTS)

sorted_COSTS = COSTS[sorted_indices]
sorted_SETS = SETS[sorted_indices]

#solution = np.full(NUM_SETS, False)
i=0

while(not valid(starting_optimized)):
    starting_optimized[i] = True
    i+=1


## Single mutation, multiple mutation and simulated annealing definition

In [39]:
def single_mutation(solution: np.ndarray)->np.ndarray:
    new_solution = solution.copy()
    i = rng.integers(0, NUM_SETS)
    new_solution[i] = not new_solution[i]
    return new_solution

def multiple_mutation(solution: np.ndarray)->np.ndarray:  
    new_solution = solution.copy()
    #The numerical value specify the probability of mutation, 0.01 = 1%. 
    mask = rng.random(NUM_SETS) < 0.1 
    new_solution = np.logical_xor(solution,mask)
    returned_solution = np.logical_or(new_solution, is_critical) #keep sets that are critical
    return returned_solution

In [40]:
get_neighbor = multiple_mutation  

def simulated_annealing(objective, solution, n_iterations, temp):
    # Initial solution
    best = solution
    best_eval = objective(best)[1]
    current, current_eval = best, best_eval

    for i in range(n_iterations):
        # Decrease temperature
        t = temp / (1 + 0.1 * i)
        
        # Generate candidate solution
        candidate = get_neighbor(current)
        candidate_eval = objective(candidate)[1]
        
        # Check if we should keep the new solution
        delta_eval = (current_eval - candidate_eval) / t
        max_exp_argument = 709  
       
        if candidate_eval > best_eval or rng.random() < math.exp(min(delta_eval, max_exp_argument)):
            current, current_eval = candidate, candidate_eval
            if valid(candidate) and candidate_eval > best_eval: 
                best, best_eval = candidate, candidate_eval

    return (best, best_eval)

## RESULTS  

In [41]:
#Simulated annealing starting from the optimized starting solution
solution_fitness = fitness(starting_optimized)
ic(solution_fitness)
solution_for_annealing = starting_optimized.copy()
(best,best_eval) = simulated_annealing(fitness, solution_for_annealing, 10000, 1)
ic(best_eval)

ic| solution_fitness: (np.True_, np.float64(-11954.831989556496))
ic| best_eval: np.float64(-11954.831989556496)


np.float64(-11954.831989556496)

In [42]:
#Simulated annealing starting from the full solution
solution_for_annealing = np.full(NUM_SETS, True)
solution_fitness = fitness(solution_for_annealing)
ic(solution_fitness)
(best,best_eval) = simulated_annealing(fitness, solution_for_annealing, 10000, 1)
ic(best_eval)

ic| solution_fitness: (np.True_, np.float64(-34139.66312253977))
ic| best_eval: np.float64(-14045.252655886718)


np.float64(-14045.252655886718)

In [43]:
#multiple tweak starting from the optimized solution
solution2 = starting_optimized.copy()
solution_fitness = fitness(solution2)
tweak = multiple_mutation
for steps in range(10_000):
    new_solution = tweak(solution2)
    new_fitness = fitness(new_solution)
    if new_fitness > solution_fitness:
        solution_fitness = new_fitness
        solution2 = new_solution
        #ic(fitness(solution))

ic(fitness(solution2))

ic| fitness(solution2): (np.True_, np.float64(-7337.505167918427))


(np.True_, np.float64(-7337.505167918427))

In [44]:
#multiple tweak starting from the full solution (all sets)
solution2 = np.full(NUM_SETS, True)
ic(fitness(solution2))
solution_fitness = fitness(solution2)
tweak = multiple_mutation
for steps in range(10_000):
    new_solution = tweak(solution2)
    new_fitness = fitness(new_solution)
    if new_fitness > solution_fitness:
        solution_fitness = new_fitness
        solution2 = new_solution
        #ic(fitness(solution))

ic(fitness(solution2))

ic| fitness(solution2): (np.True_, np.float64(-34139.66312253977))


ic| fitness(solution2): (np.True_, np.float64(-7313.036934952619))


(np.True_, np.float64(-7313.036934952619))

In [45]:
#Iterated local search starting from the optimized solution
solution_iterated = starting_optimized.copy() #start again from the optimized solution
best = solution_iterated.copy()
solution_fitness = fitness(best)
ic(solution_fitness)
for steps_perturbation in range(10):
    while True:
        solution_perturbated = multiple_mutation(solution_iterated)
        if valid(solution_perturbated):
            break
    for steps_local in range(1000):
        new_solution = single_mutation(solution_perturbated)
        fitness_new = fitness(new_solution)
        if fitness_new > solution_fitness:
            solution_fitness = fitness_new
            solution_perturbated = new_solution
            best = new_solution.copy()
ic(fitness(best))


ic| solution_fitness: (np.True_, np.float64(-11954.831989556496))
ic| fitness(best): (np.True_, np.float64(-7087.792396632931))


(np.True_, np.float64(-7087.792396632931))

In [46]:
#Iterated local search starting from the full solution (all subsets)
solution_iterated = np.full(NUM_SETS,True) 
best = solution_iterated.copy()
solution_fitness = fitness(best)
ic(solution_fitness)
for steps_perturbation in range(100):
    while True:
        solution_perturbated = multiple_mutation(solution_iterated)
        if valid(solution_perturbated):
            break
    for steps_local in range(1000):
        new_solution = single_mutation(solution_perturbated)
        fitness_new = fitness(new_solution)
        if fitness_new > solution_fitness:
            solution_fitness = fitness_new
            solution_perturbated = new_solution
            best = new_solution.copy()
ic(fitness(best))

ic| solution_fitness: (np.True_, np.float64(

-34139.66312253977))
ic| fitness(best): (np.True_, np.float64(-7168.687907867247))


(np.True_, np.float64(-7168.687907867247))